In [61]:
import pandas as pd
from ast import literal_eval

I dislike classes in python but I think it makes sense to make these classes.
I would put the adj matrix in the tfidf but that would just create confusion

In [62]:
class ProximalCues:
    def __init__(self):
        generic = lambda x: literal_eval(x)

        conv = {'pc': generic}

        self.urls = pd.read_csv("../data/urls/url-reference_new.csv", converters=conv)

    # Get the proximal cues by page id
    def get_proximal_cues_by_id(self, id):
        return self.urls.loc[self.urls.id==id].pc.iloc[0]

In [63]:
import numpy as np


class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("../data/tf_idf/tf_idf.csv")

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("../data/tf_idf/tf_idf_keywords.csv")
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

In [64]:
class AdjacencyMatrix:
    def __init__(self):
        # Loading in the adjacency matrix
        # First column and row are data immediately, so access directly by page id
        # If row(page_id_1) leads to column(page_id_2) = 1 else = 0
        self.adjacency_matrix = pd.read_csv("../data/matrices/adjacency_matrix.csv", header=None)

    # Method to get the adjacency value from two page ids
    def get_adjacency_value(self, page_id_1, page_id_2):
        return self.adjacency_matrix.iloc[page_id_1, page_id_2] == 1

In [65]:
adjacency_matrix = AdjacencyMatrix()
proximal_cues = ProximalCues()
tfidf = TfIdf()

The input to the model should be a list of documents and the order in which they were visited.
Luckily this is exactly what we have.

In [66]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('../data/clickdata/testGrouped.csv', converters=conv)

paths = df.url_id_path

We have one visit path here:

In [67]:
from pandas import DataFrame
def find_keyword_weights(path):
    weights = np.zeros(tfidf.get_number_of_keywords())
    pd.options.display.max_rows = 0
    for id in path:
        weights += tfidf.get_all_keywords_by_id_normalized(id)

    df = DataFrame(weights, columns=['weights'])
    return df.sort_values(by=['weights'], ascending=False)

Idea: for each page visited, find the weights of the keywords


In [75]:
for path in paths:
    path[:] = [x for x in path if x < 1556]
    if len(path)>5:
        path[:] = path[5:]
        top_ten_weights = find_keyword_weights(path).head(10)
        print("Keywords computed:")
        i=0
        for index in top_ten_weights.index:
            print(str(tfidf.get_keyword_by_id(index)) + " : " + str(top_ten_weights.weights.iloc[i]))
            i+=1
        print("\n")

Keywords computed:
sessie : 0.061652467180624136
helppagina : 0.03261324981846278
gegeven : 0.0325319223154668
uitloggen : 0.03165247462089657
aug : 0.03165247462089657
menu : 0.03165247462089657
uitgelogd : 0.03165247462089657
seconden : 0.03136403819220464
hoofdcontent : 0.031089022368813984
verlengen : 0.030574630764241762


Keywords computed:
aow : 0.15353037277137443
samenstellen : 0.14714911173106454
netto : 0.11649741024536589
tegelijk : 0.11236590312250572
geniet : 0.09474545334944047
plan : 0.09379977882996511
hetzelfde : 0.08281623670081512
aanvragen : 0.07339953031231955
bedrag : 0.06420918032909485
eerder : 0.061487022608054415


Keywords computed:
bericht : 8.076424575319345
laxxxxxxxxx : 1.714131400217488
dhr : 1.6548951778273846
mijnabp : 0.8185606192367599
gevonden : 0.7513311620008075
excuses : 0.6500408432954957
overzicht : 0.5034349153780793
aow : 0.49622202821933953
aanvragen : 0.47198961889129254
samenstellen : 0.4414473351931936


Keywords computed:
bericht : 1.01

C:\Users\01din\AppData\Local\Temp\ipykernel_26512\2417980606.py:35: RuntimeWarning: invalid value encountered in true_divide
  keywords_weights_normalized = keywords_weights/sum_value


Keywords computed:
aa : nan
aaa : nan
aafje : nan
aambeien : nan
aan : nan
aanbesteding : nan
aanbeveling : nan
aanbevelingen : nan
aanbevolen : nan
aanbieden : nan




KeyboardInterrupt: 

sessie
helppagina
gegeven
uitloggen
aug
menu
uitgelogd
seconden
hoofdcontent
verlengen
